<a href="https://colab.research.google.com/github/Maxime-Jasserand/ProjetIA/blob/main/Livrable_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Livrable 1 projet Leyenda


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import shutil
import pickle

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping



---

# 1. Variables d'environnement

Ici nous allons déclarer toutes nos variables globales, par exemple le nombre d'epochs, la taille des images...

In [ ]:
#Création des paramètres
image_scale = 255  # la taille des images

# le nombre de canaux de couleurs (1: pour les images noir et blanc; 3 pour les images en couleurs (rouge vert bleu) )
image_channels = 3

fit_batch_size = 32

image_shape = (image_scale, image_scale, image_channels)

fit_epochs = 30

#folder = "C:/Windows/System32/"
folder = ""

modelsPath = "Model.hdf5"



---

# 2. Chargement des données

Nous allons commencer par charger les images dans différents datasets.<br>
Nous allons diviser chaque dataset en 2 ensembles, un pour l'entrainement et un pour le test.<br>
Nous obtiendrons un ensemble de validation en utilisant validation_split lors de l'entraînement.<br>
Nous utiliserons une proportion de 80/20.


In [ ]:
#import des datasets
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
#unzip datasets ~80s pour finir
os.mkdir('/content/data_projet_IA/')
!unzip "/content/gdrive/MyDrive/IA/Dataset Livrable 1 - Photo.zip" -d "/content/data_projet_IA/initialDatasets" > /dev/null
print("Photos copiés")
!unzip "/content/gdrive/MyDrive/IA/Dataset Livrable 1 - Text.zip" -d "/content/data_projet_IA/initialDatasets" > /dev/null
print("Textes copiés")
!unzip "/content/gdrive/MyDrive/IA/Dataset Livrable 1 - Schematics.zip" -d "/content/data_projet_IA/initialDatasets" > /dev/null
print("Schémas copiés")
!unzip "/content/gdrive/MyDrive/IA/Dataset Livrable 1 - Sketch.zip" -d "/content/data_projet_IA/initialDatasets" > /dev/null
print("Esquisses copiés")
!unzip "/content/gdrive/MyDrive/IA/Dataset Livrable 1 - Painting.zip" -d "/content/data_projet_IA/initialDatasets" > /dev/null
print("Peintures copiés")

Photos copiés
Textes copiés
Schémas copiés
Esquisses copiés
Peintures copiés


In [ ]:
#[CREATION TRAIN SET] ~120s pour finir

#copier les fichiers vers le train set
shutil.copytree(folder + '/content/data_projet_IA/initialDatasets', 
                folder + '/content/data_projet_IA/train_set', 
                symlinks=False, ignore=None,
                ignore_dangling_symlinks=False)

'/content/data_projet_IA/train_set'

In [ ]:
#récupérer la taille des dossiers
path, dirs, files = next(os.walk(folder + '/content/data_projet_IA/train_set/Photo'))
path2, dirs2, files2 = next(os.walk(folder + '/content/data_projet_IA/train_set/Sketch'))

#move 20% images to validation
os.mkdir(folder + '/content/data_projet_IA/validation_set/')
os.mkdir(folder + '/content/data_projet_IA/validation_set/Photo/')
os.mkdir(folder + '/content/data_projet_IA/validation_set/Text/')
os.mkdir(folder + '/content/data_projet_IA/validation_set/Schematics/')
os.mkdir(folder + '/content/data_projet_IA/validation_set/Sketch/')
os.mkdir(folder + '/content/data_projet_IA/validation_set/Painting/')

photoList = os.listdir(folder + '/content/data_projet_IA/train_set/Photo')
textList = os.listdir(folder + '/content/data_projet_IA/train_set/Text')
schematicsList = os.listdir(folder + '/content/data_projet_IA/train_set/Schematics')
sketchList = os.listdir(folder + '/content/data_projet_IA/train_set/Sketch')
paintingList = os.listdir(folder + '/content/data_projet_IA/train_set/Painting')

for i in range(0,int(len(files)/5)):
  shutil.move(folder + "/content/data_projet_IA/train_set/Photo/" + photoList[i], folder + "/content/data_projet_IA/validation_set/Photo/" + photoList[i])
  shutil.move(folder + "/content/data_projet_IA/train_set/Text/" + textList[i], folder + "/content/data_projet_IA/validation_set/Text/" + textList[i])
  shutil.move(folder + "/content/data_projet_IA/train_set/Schematics/" + schematicsList[i], folder + "/content/data_projet_IA/validation_set/Schematics/" + schematicsList[i])
  shutil.move(folder + "/content/data_projet_IA/train_set/Painting/" + paintingList[i], folder + "/content/data_projet_IA/validation_set/Painting/" + paintingList[i])
for i in range(0,int(len(files2)/5)):
  shutil.move(folder + "/content/data_projet_IA/train_set/Sketch/" + sketchList[i], folder + "/content/data_projet_IA/validation_set/Sketch/" + sketchList[i])



---

# 3. Traitement des données

Pour pouvoir exploiter nos données nous allons devoir leur appliquer un certain nombre de pré-traitements :
*  Elles devront toutes avoir la même taille pour pouvoir être exploitée par le CNN.

*  Afin de réduire l'overfitting nous allons effectuer une Data Augmentation.

*  Nous allons garder un set sans Data Augmentation pour comparer les résultats.

In [ ]:
# training_data_generator: charge les données d'entrainement en mémoire
# quand il charge les images, il les ajuste (change la taille, les dimensions, la direction ...)
# aléatoirement afin de rendre le modèle plus robuste à la position du sujet dans les images
# Note: On peut utiliser cette méthode pour augmenter le nombre d'images d'entrainement (data augmentation)
training_data_generator = ImageDataGenerator(
    rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
    rescale=1. / 255,
    shear_range=0.1,
    horizontal_flip=True)

# validation_data_generator: charge les données de validation en memoire
validation_data_generator = ImageDataGenerator(rescale=1. / 255)

# training_generator: indique la méthode de chargement des données d'entrainement
training_generator = training_data_generator.flow_from_directory(
    folder + '/content/data_projet_IA/train_set',  # Place des images d'entrainement
    color_mode='rgb',  # couleur des images
    target_size=(image_scale, image_scale),  # taille des images
    batch_size=fit_batch_size,  # nombre d'images à entrainer (batch size)
    class_mode="categorical",  # classement binaire (problème de 2 classes)
    shuffle=True)  # on "brasse" (shuffle) les données -> pour prévenir le surapprentissage

# validation_generator: indique la méthode de chargement des données de validation
validation_generator = validation_data_generator.flow_from_directory(
    folder + '/content/data_projet_IA/validation_set',  # Place des images de validation
    color_mode='rgb',  # couleur des images
    target_size=(image_scale, image_scale),  # taille des images
    batch_size=fit_batch_size,  # nombre d'images à valid
    class_mode="categorical",  # classement binaire (problème de 2 classes)
    shuffle=True)  # on "brasse" (shuffle) les données -> pour prévenir le surapprentissage

# On imprime l'indice de chaque classe (Keras numerote les classes selon l'ordre des dossiers des classes)
# Dans ce cas => [2: 0 et 7:1]
print(training_generator.class_indices)
print(validation_generator.class_indices)

# On charge les données d'entrainement et de validation
# x_train: Les données d'entrainement
# y_train: Les Ètiquettes des données d'entrainement
# x_val: Les données de validation
# y_val: Les Ètiquettes des données de validation
(x_train, y_train) = training_generator.next()
(x_val, y_val) = validation_generator.next()

Found 33125 images belonging to 5 classes.
Found 8281 images belonging to 5 classes.
{'Painting': 0, 'Photo': 1, 'Schematics': 2, 'Sketch': 3, 'Text': 4}
{'Painting': 0, 'Photo': 1, 'Schematics': 2, 'Sketch': 3, 'Text': 4}




---

# 4. Déclaration des modèles

Ensuite nos allons construire nos différents modèles.<br>
Nous avons choisi d'utiliser 3 modèles :


*   Un premier modèle, très simple, composé de 2 couches de convolutions et 2 couches denses.
*   Un modèle avec 2 fois plus de couches de convolution.
*   Un modèle avec une architecture reconnue, VGG-16.

L'idée est de pouvoir comparer différents modèles afin de visualiser en quoi la profondeur va influencer la précision et le temps d'apprentissage.





In [ ]:
def feature_extraction(input):

    # 1-couche de convolution avec nombre de filtre  (exp 32)  avec la taille de la fenetre de ballaiage exp : 3x3
    # 2-fonction d'activation exp: sigmoid, relu, tanh ...
    # 3-couche d'echantillonage (pooling) pour reduire la taille avec la taille de la fenetre de ballaiage exp :2x2

    # **** On répète ces étapes tant que nécessaire ****

    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(input)
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
    x = Dense(128, activation='relu')
    x = Dense(128, activation='relu')

    return x

def fully_connected(encoded):
# Flatten: pour convertir les matrices en vecteurs pour la couche MLP
# Dense: une couche neuronale simple avec le nombre de neurone (exemple 64)
# fonction d'activation exp: sigmoid, relu, tanh ...
  x = Flatten(input_shape=(255,255,3))(encoded)

# Puisque'on a une classification binaire, la dernière couche doit être formée d'un seul neurone avec une fonction d'activation sigmoide
# La fonction sigmoide nous donne une valeur entre 0 et 1
# On considère les résultats <=0.5 comme l'image appartenant à la classe 0 (c.-à-d. la classe qui correspond au chiffre 2)
# on considère les résultats >0.5 comme l'image appartenant à la classe 0 (c.-à-d. la classe qui correspond au chiffre 7)
  x = Dense(5)(x)
  sortie = Activation('softmax')(x)
  return sortie

In [ ]:
"""model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=5,
    classifier_activation="softmax",
)"""



---

# 5. Entraînement

Il est temps d'entraîner nos modèles.

In [ ]:
#compilation du modèle 
model.compile(loss='categorical_crossentropy',
              optimizer="RMSProp", metrics=['accuracy'])

modelcheckpoint = ModelCheckpoint(filepath=modelsPath,
                                  monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=10, verbose=1, mode='auto')

In [1]:
visualkeras.layered_view(model, to_file='output.png', legend = True).show() # write and show model architecture

NameError: ignored

In [ ]:
#entrainement du modèle 
classifier = model.fit(training_generator,
                       epochs=10,  # nombre d'époques
                       batch_size=fit_batch_size,  # nombre d'images entrainées ensemble
                       validation_data=validation_generator,  # données de validation
                       verbose=1,  # mets cette valeur ‡ 0, si vous voulez ne pas afficher les détails d'entrainement
                       shuffle=True,
                       max_queue_size=15, 
                       # les fonctions à appeler à la fin de chaque époque (dans ce cas modelcheckpoint: qui sauvegarde le modèle)
                       callbacks=[modelcheckpoint, early],
                       workers = 4)  # shuffle les images

NameError: ignored



---

# 6. Tester les modèles

Nous allons appliquer nos modèles sur les données de test afin de nous assurer de leur précision.<br>
Nous allons également afficher les matrices de confusion.



---

# 7. Influence des hyperparamètres

Pour tenter d'améliorer nos modèles nous allons effectuer de l'hypertuning : en faisant varier certains hyperparamètres nous allons peut-être obtenir des modèles plus rapides ou plus précis.
Nous ne cherchons pas à trouver le modèle le plus précis mais simplement a visualiser l'influence des hyperparamètres.



---

# 8. Transfert learning

Dans l'optique d'obtenir le modèle le plus précis possible nous allons nous servir d'un modèle pré-entraîné.




---

# 9. Comparer les résultats



---

# 10. Conclusion